#Create Convolutional Neural Network and Optimize it using Keras Tuner

###Importing Libraries

In [14]:
#Install Keras
!pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [16]:
print("TensorFlow Version : ",tf.__version__)
print("Keras version : ",keras.__version__)

TensorFlow Version :  2.2.0-rc4
Keras version :  2.3.0-tf


###Importing Dataset

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

In [18]:
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.6/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

In [19]:
(train_images,train_labels) , (test_images,test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [20]:
print("Train Labels : ",train_labels,"\n\n Train Data : \n",train_images)

Train Labels :  [9 0 0 ... 3 0 5] 

 Train Data : 
 [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


###Data Pre-processing

In [0]:
#Standard Scaling
train_images = train_images/255.0
test_images = test_images/255.0

In [22]:
train_images[0].shape

(28, 28)

In [0]:
#Reshape the data for the model fit
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

### Building Model

In [0]:
#Function for hyper parameters model [ you can get official documentation of this hyperparameter tuning here : https://keras-team.github.io/keras-tuner/]
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('Convolutional_Filter_1',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('Convolutional_Kernal_1',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('Convolutional_Filter_2',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('Convolutional_Kernal_1',values=[3,5]),
        activation='relu',
    ),                          
    keras.layers.Flatten(),

    keras.layers.Dense(
        units=hp.Int('Dense_unit_1',min_value=32,max_value=128,step=16),
        activation='relu'
    ),

    keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [29]:
tuner

In [30]:
tuner.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4454 - accuracy: 0.8409 - val_loss: 0.3339 - val_accuracy: 0.8775
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3157 - accuracy: 0.8836 - val_loss: 0.3137 - val_accuracy: 0.8872
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2839 - accuracy: 0.8942 - val_loss: 0.3437 - val_accuracy: 0.8825


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4171 - accuracy: 0.8475 - val_loss: 0.3131 - val_accuracy: 0.8812
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2756 - accuracy: 0.8972 - val_loss: 0.2686 - val_accuracy: 0.9002
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2231 - accuracy: 0.9180 - val_loss: 0.2597 - val_accuracy: 0.9080


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4058 - accuracy: 0.8514 - val_loss: 0.3056 - val_accuracy: 0.8870
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2641 - accuracy: 0.9029 - val_loss: 0.2659 - val_accuracy: 0.8995
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2106 - accuracy: 0.9206 - val_loss: 0.2506 - val_accuracy: 0.9093


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4058 - accuracy: 0.8539 - val_loss: 0.3027 - val_accuracy: 0.8873
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2673 - accuracy: 0.9021 - val_loss: 0.2624 - val_accuracy: 0.9035
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2122 - accuracy: 0.9215 - val_loss: 0.2525 - val_accuracy: 0.9097


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3911 - accuracy: 0.8589 - val_loss: 0.2865 - val_accuracy: 0.8967
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2496 - accuracy: 0.9089 - val_loss: 0.2557 - val_accuracy: 0.9088
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1835 - accuracy: 0.9321 - val_loss: 0.2593 - val_accuracy: 0.9153


INFO:tensorflow:Oracle triggered exit


In [0]:
model = tuner.get_best_models(num_models=1)[0]

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1769536   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,812,666
Trainable params: 1,812,666
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1332 - accuracy: 0.9510 - val_loss: 0.2825 - val_accuracy: 0.9152
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0925 - accuracy: 0.9655 - val_loss: 0.2897 - val_accuracy: 0.9080
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0618 - accuracy: 0.9775 - val_loss: 0.3600 - val_accuracy: 0.9087
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0459 - accuracy: 0.9827 - val_loss: 0.4025 - val_accuracy: 0.9130
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0357 - accuracy: 0.9875 - val_loss: 0.4780 - val_accuracy: 0.9082
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0267 - accuracy: 0.9906 - val_loss: 0.4721 - val_accuracy: 0.9063
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0223 - accuracy: 0.9921 - val_loss: 0.5035 - val_accuracy

### Bonus


In [36]:
#Know your GPU details
tf.test.gpu_device_name()

'/device:GPU:0'

In [38]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()  #  Tesla P100-PCIE-16GB will cost you 9.5k dollar

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3292422363448439110, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7595788407118915143
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2932291546940592508
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15701463552
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6104047378273420596
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]